In [7]:
import pandas as pd

# --- Rutas de entrada ---
path_prod00 = "~/buckets/b1/exp/prod_00/predicciones/pred_mod_1.csv"
path_prod02 = "~/buckets/b1/exp/prod_02/predicciones/pred_mod_1.csv"

# --- Leer archivos ---
df00 = pd.read_csv(path_prod00)
df02 = pd.read_csv(path_prod02)

# --- Verificaciones básicas ---
# Ambos deben tener las columnas: numero_de_cliente, prob
assert "numero_de_cliente" in df00.columns
assert "prob" in df00.columns
assert "numero_de_cliente" in df02.columns
assert "prob" in df02.columns

# --- Guardar los 12k mejores individuales ---
top_12k_00 = df00.nlargest(12000, "prob")["numero_de_cliente"]
top_12k_02 = df02.nlargest(12000, "prob")["numero_de_cliente"]

top_12k_00.to_csv("preds_prod00.csv", index=False, header=False)
top_12k_02.to_csv("preds_prod02.csv", index=False, header=False)

# --- Combinar por numero_de_cliente ---
# Hago merge para alinear predicciones del mismo cliente
df = pd.merge(df00[["numero_de_cliente", "prob"]],
              df02[["numero_de_cliente", "prob"]],
              on="numero_de_cliente",
              suffixes=("_00", "_02"))

# --- Promedio ---
df["prob_prom"] = (df["prob_00"] + df["prob_02"]) / 2

# --- Top 12k del ensemble ---
top_12k_ensemble = df.nlargest(12000, "prob_prom")["numero_de_cliente"]

# --- Guardar archivo final sin header ---
top_12k_ensemble.to_csv("preds_ensemble.csv", index=False, header=False)

print("Archivos generados:")
print(" - preds_prod00.csv")
print(" - preds_prod02.csv")
print(" - preds_ensemble.csv")


Archivos generados:
 - preds_prod00.csv
 - preds_prod02.csv
 - preds_ensemble.csv


In [8]:
# --- Leer archivos (sin header) ---
prod00 = pd.read_csv("preds_prod00.csv", header=None)[0]
prod02 = pd.read_csv("preds_prod02.csv", header=None)[0]
ensemble = pd.read_csv("preds_ensemble.csv", header=None)[0]

# Convertir a conjuntos para operaciones rápidas
set00 = set(prod00)
set02 = set(prod02)
set_ens = set(ensemble)

# --- Cálculo de intersecciones ---
inter_3 = set00 & set02 & set_ens
inter_ens_00 = set_ens & set00
inter_ens_02 = set_ens & set02
inter_00_02 = set00 & set02

# --- Mostrar resultados ---
print("Clientes en común entre las tres predicciones:", len(inter_3))
print("Clientes en común entre ensemble y prod00:", len(inter_ens_00))
print("Clientes en común entre ensemble y prod02:", len(inter_ens_02))
print("Clientes en común entre prod00 y prod02:", len(inter_00_02))

Clientes en común entre las tres predicciones: 10817
Clientes en común entre ensemble y prod00: 11224
Clientes en común entre ensemble y prod02: 11093
Clientes en común entre prod00 y prod02: 10818
